In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import tensorflow as tf
from pprint import pprint
import cv2

2022-11-27 12:17:16.432357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-27 12:17:16.432421: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [59]:
ROOT_DIR = os.path.abspath('.')
MODELS_DIR = os.path.join(ROOT_DIR, 'models')
CUSTOM_MODEL = os.path.join(MODELS_DIR, 'experiments', 'full_int8.tflite')
CUSTOM_MODEL_2 = os.path.join(MODELS_DIR, 'experiments', 'integer_only.tflite')
CUSTOM_MODEL_3 = os.path.join(MODELS_DIR, 'experiments', 'integer_only2.tflite')
CUSTOM_MODEL_4 = os.path.join(MODELS_DIR, 'experiments', 'int8.tflite')
# IMAGE_DIR =  os.path.join(ROOT_DIR, 'custom_dataset')
IMAGE_DIR =  os.path.join(ROOT_DIR, 'test')


In [3]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.int8'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (3.075740460189991e-05, -1),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([3.0757405e-05], dtype=float32),
                             'zero_points': array([-1], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
{'dtype': <class 'numpy.int8'>,
 'index': 386,
 'name': 'StatefulPartitionedCall:1',
 'quantization': (0.00028331970679573715, -128),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([0.00028332], dtype=float32),
                             'zero_points': array([-128], dtype=int32)},
 'shape': array([ 1, 10], dtype=int32),
 'shape_signature': array([ 1, 10], dtype=int32),
 'sparsity_parameters': {}}


In [95]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'DSQ-6618 (2).jpg'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
test_image = np.array(img)
test_image

array([[[ 92,  88,  86],
        [197, 184, 170],
        [252, 235, 212],
        ...,
        [179, 193, 193],
        [179, 191, 190],
        [173, 183, 182]],

       [[ 71,  68,  66],
        [187, 174, 159],
        [252, 236, 212],
        ...,
        [178, 190, 190],
        [179, 189, 188],
        [176, 187, 183]],

       [[ 74,  70,  68],
        [186, 173, 159],
        [249, 231, 207],
        ...,
        [177, 189, 188],
        [176, 185, 183],
        [175, 184, 181]],

       ...,

       [[ 93,  93,  93],
        [ 73,  73,  73],
        [ 99,  99,  99],
        ...,
        [162, 141, 119],
        [185, 159, 136],
        [180, 152, 128]],

       [[105, 105, 105],
        [ 64,  64,  64],
        [ 96,  96,  96],
        ...,
        [161, 140, 119],
        [172, 148, 125],
        [199, 171, 147]],

       [[ 72,  72,  72],
        [ 99,  99,  99],
        [ 87,  87,  87],
        ...,
        [143, 122, 101],
        [157, 134, 110],
        [169, 143, 118]]

In [96]:
input_scale, input_zero_point = input_details["quantization"]
test_image = test_image / input_scale + input_zero_point
test_image

array([[[2991148.64967873, 2861098.66491009, 2796073.67252577],
        [6404960.74985555, 5982298.29935747, 5527123.35266723],
        [8193148.04042436, 7640435.60515764, 6892648.19273795],
        ...,
        [5819735.81839667, 6274910.76508691, 6274910.76508691],
        [5819735.81839667, 6209885.77270259, 6177373.27651043],
        [5624660.84124371, 5949785.80316531, 5917273.30697315]],

       [[2308386.22964337, 2210848.74106689, 2145823.74868257],
        [6079835.78793395, 5657173.33743587, 5169485.89455346],
        [8193148.04042436, 7672948.1013498 , 6892648.19273795],
        ...,
        [5787223.32220451, 6177373.27651043, 6177373.27651043],
        [5819735.81839667, 6144860.78031827, 6112348.28412611],
        [5722198.32982019, 6079835.78793395, 5949785.80316531]],

       [[2405923.71821985, 2275873.73345121, 2210848.74106689],
        [6047323.29174179, 5624660.84124371, 5169485.89455346],
        [8095610.55184788, 7510385.620389  , 6730085.71177715],
        ..

In [97]:
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
test_image

array([[[[ 44,  42,  41],
         [ 96,  90,  83],
         [124, 115, 104],
         ...,
         [ 87,  94,  94],
         [ 87,  93,  93],
         [ 84,  89,  89]],

        [[ 34,  32,  31],
         [ 91,  85,  77],
         [124, 116, 104],
         ...,
         [ 87,  93,  93],
         [ 87,  92,  92],
         [ 86,  91,  89]],

        [[ 35,  33,  32],
         [ 91,  84,  77],
         [122, 113, 101],
         ...,
         [ 86,  92,  92],
         [ 86,  90,  89],
         [ 85,  90,  88]],

        ...,

        [[ 45,  45,  45],
         [ 35,  35,  35],
         [ 48,  48,  48],
         ...,
         [ 79,  68,  58],
         [ 90,  77,  66],
         [ 88,  74,  62]],

        [[ 51,  51,  51],
         [ 30,  30,  30],
         [ 46,  46,  46],
         ...,
         [ 78,  68,  58],
         [ 84,  72,  61],
         [ 97,  83,  71]],

        [[ 34,  34,  34],
         [ 48,  48,  48],
         [ 42,  42,  42],
         ...,
         [ 69,  59,  49],
        

In [102]:
interpreter.set_tensor(input_details["index"], test_image)

In [103]:
interpreter.invoke()

In [104]:
def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))

In [106]:
scores

# array([120,  24,  10,  -4, -18, -31, -45, -59, -59, -59], dtype=int8)

array([120,  10,  10,  -4, -31, -45, -45, -59, -59, -59], dtype=int8)

In [107]:
output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.07026328728534281

In [110]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars11.png'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
test_image = np.array(img)
input_scale, input_zero_point = input_details["quantization"]
test_image = test_image / input_scale + input_zero_point
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])


In [115]:
interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()

In [116]:
boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))

In [117]:
output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.07026328728534281

In [133]:
output_scale, output_zero_point  = interpreter.get_output_details()[3]['quantization']
output = (boxes[0] - output_zero_point) * output_scale
output

array([-6.90196039e-07, -7.05882313e-07, -5.33333303e-07,  2.11764694e-07])

#### LAST DEBUG

In [5]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'LBS-2737.jpg'))
img[:1]

array([[[111, 111, 111],
        [117, 117, 117],
        [119, 119, 119],
        ...,
        [107, 107, 101],
        [113, 113, 107],
        [129, 126, 121]]], dtype=uint8)

In [6]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars11.png'))
img[:1]

array([[[40, 39, 36],
        [43, 41, 38],
        [46, 42, 39],
        ...,
        [30, 27, 27],
        [34, 29, 30],
        [47, 43, 43]]], dtype=uint8)

In [7]:
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
img[:1]

array([[[ 36,  39,  40],
        [ 38,  41,  44],
        [ 72,  74,  78],
        [137, 138, 142],
        [139, 139, 140],
        [143, 143, 139],
        [143, 144, 136],
        [143, 144, 135],
        [142, 143, 134],
        [143, 142, 134],
        [146, 144, 137],
        [144, 143, 138],
        [143, 145, 139],
        [139, 144, 137],
        [132, 138, 128],
        [121, 126, 118],
        [101, 105,  98],
        [ 95,  99,  93],
        [ 97, 101,  95],
        [101, 105,  99],
        [100, 104, 100],
        [ 92,  93,  90],
        [ 91,  90,  89],
        [113, 112, 110],
        [113, 112, 110],
        [ 97,  96,  92],
        [ 79,  78,  73],
        [ 77,  76,  72],
        [ 67,  68,  62],
        [ 66,  67,  61],
        [ 73,  74,  69],
        [ 67,  68,  63],
        [ 67,  68,  63],
        [ 72,  72,  67],
        [ 69,  69,  64],
        [ 69,  71,  63],
        [ 73,  75,  67],
        [ 76,  78,  72],
        [ 78,  80,  75],
        [ 77,  79,  76],


In [8]:
test_image = np.array(img)
test_image[:1]

array([[[ 36,  39,  40],
        [ 38,  41,  44],
        [ 72,  74,  78],
        [137, 138, 142],
        [139, 139, 140],
        [143, 143, 139],
        [143, 144, 136],
        [143, 144, 135],
        [142, 143, 134],
        [143, 142, 134],
        [146, 144, 137],
        [144, 143, 138],
        [143, 145, 139],
        [139, 144, 137],
        [132, 138, 128],
        [121, 126, 118],
        [101, 105,  98],
        [ 95,  99,  93],
        [ 97, 101,  95],
        [101, 105,  99],
        [100, 104, 100],
        [ 92,  93,  90],
        [ 91,  90,  89],
        [113, 112, 110],
        [113, 112, 110],
        [ 97,  96,  92],
        [ 79,  78,  73],
        [ 77,  76,  72],
        [ 67,  68,  62],
        [ 66,  67,  61],
        [ 73,  74,  69],
        [ 67,  68,  63],
        [ 67,  68,  63],
        [ 72,  72,  67],
        [ 69,  69,  64],
        [ 69,  71,  63],
        [ 73,  75,  67],
        [ 76,  78,  72],
        [ 78,  80,  75],
        [ 77,  79,  76],


In [10]:
input_scale, input_zero_point = input_details["quantization"]
test_image = (test_image / input_scale) + input_zero_point
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
test_image[:1]

array([[[[[ 6,  7,  7],
          [ 7,  8,  8],
          [15, 16, 17],
          ...,
          [ 4,  4,  4],
          [ 5,  4,  5],
          [ 8,  8,  9]],

         [[ 7,  7,  8],
          [ 7,  8,  9],
          [16, 16, 17],
          ...,
          [ 4,  4,  4],
          [ 5,  4,  5],
          [ 7,  7,  8]],

         [[ 8,  8,  9],
          [ 9, 10, 10],
          [16, 17, 17],
          ...,
          [ 6,  6,  6],
          [ 5,  5,  5],
          [ 5,  4,  5]],

         ...,

         [[ 7,  7,  7],
          [ 8,  8,  7],
          [ 7,  7,  7],
          ...,
          [ 7,  7,  7],
          [ 7,  7,  6],
          [ 6,  6,  5]],

         [[ 1,  1,  1],
          [ 1,  1,  1],
          [ 1,  1,  1],
          ...,
          [ 1,  1,  1],
          [ 1,  1,  1],
          [ 1,  1,  0]],

         [[-1, -1, -1],
          [-1, -1, -1],
          [-1, -1, -1],
          ...,
          [-1, -1, -1],
          [-1, -1, -1],
          [-1, -1, -1]]]]], dtype=int8)

In [13]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.int8'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (3.075740460189991e-05, -1),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([3.0757405e-05], dtype=float32),
                             'zero_points': array([-1], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
[{'dtype': <class 'numpy.int8'>,
  'index': 386,
  'name': 'StatefulPartitionedCall:1',
  'quantization': (0.00028331970679573715, -128),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00028332], dtype=float32),
                              'zero_points': array([-128], dtype=int32)},
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([ 1, 10], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': <class 'numpy.int8'>,
  'index': 384,
  'name': 'S

In [26]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL_2)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.int8'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (0.007843137718737125, -1),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([0.00784314], dtype=float32),
                             'zero_points': array([-1], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
[{'dtype': <class 'numpy.int8'>,
  'index': 386,
  'name': 'StatefulPartitionedCall:1',
  'quantization': (0.0038589821197092533, -128),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00385898], dtype=float32),
                              'zero_points': array([-128], dtype=int32)},
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([ 1, 10], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': <class 'numpy.int8'>,
  'index': 384,
  'name': 'Statef

In [27]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars11.png'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))

In [28]:
# tensor_index = interpreter.get_input_details()[0]['index']
# input_tensor = interpreter.tensor(tensor_index)()[0]
# input_tensor[:, :] = np.expand_dims((img-255)/255, axis=0)

test_image = np.array(img)
input_scale, input_zero_point = input_details["quantization"]
test_image = test_image / input_scale + input_zero_point
test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
test_image

array([[[[-20, 107, -22],
         [-21, 106, -24],
         [-38, -39, -41],
         ...,
         [-15, -15, 113],
         [112, -16, -18],
         [106, 106, 104]],

        [[108, -22, 106],
         [-22, 105, -25],
         [ 90,  89,  87],
         ...,
         [113, 113, -16],
         [112, -16, -18],
         [107, -21, -23]],

        [[106, -24, 104],
         [103, -27, -28],
         [-39,  88, -42],
         ...,
         [110, 110, 110],
         [-17, -17, 111],
         [112, -16, 111]],

        ...,

        [[-22, -22, -21],
         [106, 106, 107],
         [107, 107, 108],
         ...,
         [107, 107, -21],
         [108, 108, 109],
         [110, 110, 111]],

        [[ -9,  -9, 120],
         [119, 119,  -9],
         [119, 119,  -9],
         ...,
         [119, 119,  -9],
         [ -9,  -9, 120],
         [ -9,  -9,  -8]],

        [[126, 126,  -1],
         [ -1,  -1,  -1],
         [ -1,  -1,  -1],
         ...,
         [126, 126,  -1],
        

In [29]:
interpreter.set_tensor(input_details["index"], test_image)

In [30]:
interpreter.invoke()

In [31]:
def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))

In [32]:
scores

array([-112, -118, -119, -119, -120, -121, -122, -122, -122, -122],
      dtype=int8)

In [34]:
scores[scores.argmax()]

-112

In [37]:
pprint(interpreter.get_output_details()[0])

{'dtype': <class 'numpy.int8'>,
 'index': 386,
 'name': 'StatefulPartitionedCall:1',
 'quantization': (0.0038589821197092533, -128),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([0.00385898], dtype=float32),
                             'zero_points': array([-128], dtype=int32)},
 'shape': array([ 1, 10], dtype=int32),
 'shape_signature': array([ 1, 10], dtype=int32),
 'sparsity_parameters': {}}


In [33]:
output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.06174371391534805

In [40]:
input_scale, _ = input_details['quantization']

In [41]:
input_scale * output_zero_point / output_scale

-260.1519252631288

### Integer Only 2

In [44]:
interpreter = tf.lite.Interpreter(CUSTOM_MODEL_3)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()
pprint(input_details)
pprint(output_details)

{'dtype': <class 'numpy.int8'>,
 'index': 0,
 'name': 'serving_default_input:0',
 'quantization': (0.003921568859368563, -128),
 'quantization_parameters': {'quantized_dimension': 0,
                             'scales': array([0.00392157], dtype=float32),
                             'zero_points': array([-128], dtype=int32)},
 'shape': array([  1, 320, 320,   3], dtype=int32),
 'shape_signature': array([  1, 320, 320,   3], dtype=int32),
 'sparsity_parameters': {}}
[{'dtype': <class 'numpy.int8'>,
  'index': 386,
  'name': 'StatefulPartitionedCall:1',
  'quantization': (0.00385795789770782, -128),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00385796], dtype=float32),
                              'zero_points': array([-128], dtype=int32)},
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([ 1, 10], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': <class 'numpy.int8'>,
  'index': 384,
  'name': 'Stat

In [54]:
img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars11.png'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
test_image = np.array(img)

In [58]:
input_scale, input_zero_point = input_details["quantization"]
test_image = (test_image / input_scale) + input_zero_point

test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
test_image

array([[[[[  91,   88,   87],
          [  89,   86,   83],
          [  55,   53,   49],
          ...,
          [ 101,  101,  100],
          [  98,   99,   95],
          [  86,   86,   82]],

         [[  90,   87,   86],
          [  87,   84,   81],
          [  54,   52,   48],
          ...,
          [ 100,  100,   99],
          [  98,   99,   95],
          [  88,   89,   85]],

         [[  86,   83,   82],
          [  80,   77,   75],
          [  53,   50,   47],
          ...,
          [  94,   94,   94],
          [  97,   97,   96],
          [  98,   99,   96]],

         ...,

         [[  87,   87,   89],
          [  86,   86,   88],
          [  88,   88,   90],
          ...,
          [  88,   88,   89],
          [  90,   90,   92],
          [  94,   94,   96]],

         [[ 113,  113,  114],
          [ 112,  112,  113],
          [ 112,  112,  113],
          ...,
          [ 112,  112,  113],
          [ 113,  113,  114],
          [ 113,  113,  115]],



In [56]:
def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()
boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))

In [57]:
output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.2932048002257943

In [47]:
output_scale, output_zero_point  = interpreter.get_output_details()[0]['quantization']
output = (scores[scores.argmax()] - output_zero_point) * output_scale
output

0.5439720635768026

### Full integer

In [60]:
CUSTOM_MODEL_3 = os.path.join(MODELS_DIR, 'experiments', 'integer_only2.tflite')

img = cv2.imread(os.path.join(IMAGE_DIR, 'Cars11.png'))
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']
img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (input_width, input_height))
test_image = np.array(img)

input_scale, input_zero_point = input_details["quantization"]
test_image = (test_image / input_scale) + input_zero_point

test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])

def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor


interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()
boxes = get_output_tensor(interpreter, 1)
classes = get_output_tensor(interpreter, 3)
scores = get_output_tensor(interpreter, 0)
count = int(get_output_tensor(interpreter, 2))